In [93]:
from pathlib import Path
import pandas as pd
import random
import stockInfo
import rceptnoInfo
import report
import requests
from bs4 import BeautifulSoup
import re

In [17]:
rcept_no = '20211115001521'
# rcept_no = '20210309000744'
# rcept_no = '20121129001089'

In [18]:
url = f'http://dart.fss.or.kr/dsaf001/main.do?rcpNo={rcept_no}'
r = requests.get(url)
reportHtml = r.text

In [19]:
parser1 = report.Parser(nord = report.Nord2Parser(reportHtml), parser_format = r'.*연결재무제표$')
parser2 = report.Parser(nord = report.Nord1Parser(reportHtml), parser_format = r'.*재무제표 등$')
parser3 = report.Parser(nord = report.Nord2Parser(reportHtml), parser_format = r'^[^가-힣]*재무제표$')

In [20]:
dic = parser1.operation()

In [122]:
url = f'http://dart.fss.or.kr/report/viewer.do?'
r = requests.get(url, params=dic[0])
html = r.text
soup = BeautifulSoup(html, 'html.parser')

In [25]:
pd.read_html(html)[0]

,0
0,제40기 3분기 2021년 9월 30일 현재
1,제39기 2020년 12월 31일 현재
2,제38기 2019년 12월 31일 현재


In [39]:
soup.html.body.p.next_sibling.next_sibling.string

'2. 연결재무제표'

In [119]:
step1 = soup.find_all('span', string=re.compile('연결재무상태표'))[0]
step2 = step1.parent
step3 = step2.next_sibling
step4 = step3.next_sibling
step5 = step4.next_sibling
step6 = step5.next_sibling
step7 = step6.next_sibling
step8 = step7.next_sibling
step8.find_all('td', string=re.compile('^[^가-힣]*자\s*본\s*총\s*계'))
# soup.find_all('span', string='가. 연결재무상태표')[0].parent.next_sibling.next_sibling.next_sibling.next_sibling.next_sibling.next_sibling

[<td align="CENTER" height="20" valign="TOP" width="301">자     본     총     계</td>]

In [170]:
class Handler:
    parser = r'연결재무상태표'
    def __init__(self, soup):
        self.soup = soup
    def request_handler(self):
        lst = self.soup.find_all('span', string=re.compile(self.parser))
        print(lst)
        if lst:
            print('here')
            soup = lst[0].parent
            print(soup)
            lst = False
            while not lst :
                print('while')
                soup = soup.next_sibling
                print(f'!!{soup.string}')
                print(f'!!{len(soup.string)}')
                if len(soup.string)>1:
                    lst = soup.find_all('td', string=re.compile('^[^가-힣]*자\s*본\s*총\s*계'))
            return lst[0]

In [171]:
Handler(soup).request_handler()

[<span>가. 연결재무상태표</span>]
here
<p><span>가. 연결재무상태표</span><span></span><br/>
</p>
while
!!

!!1
while
!!None


TypeError: object of type 'NoneType' has no len()